In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import os

In [2]:
train_files = pd.read_csv(r"C:\Users\bhara\OneDrive\Desktop\Ship Classification\train.csv")

In [3]:
train_files.head()

,image,category
0,2823080.jpg,1
1,2870024.jpg,1
2,2662125.jpg,2
3,2900420.jpg,3
4,2804883.jpg,2


In [4]:
ship = {1: 'Cargo',
        2: 'Military',
        3: 'Carrier',
        4: 'Cruise',
        5: 'Tankers'}

In [5]:
train_files['ship']=train_files['category'].map(ship).astype('category')

In [6]:
train_files.head()

,image,category,ship
0,2823080.jpg,1,Cargo
1,2870024.jpg,1,Cargo
2,2662125.jpg,2,Military
3,2900420.jpg,3,Carrier
4,2804883.jpg,2,Military


In [8]:
labels = train_files.sort_values('ship')
class_names = list(labels.ship.unique())
for i in class_names:
  os.makedirs(os.path.join(r"C:\Users\bhara\OneDrive\Desktop\Ship Classification\input\train",i))

In [9]:
import shutil
for c in class_names:
  for i in list(labels[labels['ship']==c]['image']):
    get_image = os.path.join(r"C:\Users\bhara\OneDrive\Desktop\Ship Classification\input\images", i)
    move_image_to_cat = shutil.move(get_image, r"C:\Users\bhara\OneDrive\Desktop\Ship Classification\input\train", c)

In [10]:
!pip install tensorflow

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [8]:
train_datagen = ImageDataGenerator(rotation_range=45,
                                  horizontal_flip=True,
                                  height_shift_range=0.5,
                                  validation_split=0.2,
                                  preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [10]:
train_set = train_datagen.flow_from_directory(r"Ship Classification\input\train",
                                             batch_size=16, subset='training',
                                             target_size=(224,224))
validation_set = train_datagen.flow_from_directory(r"Ship Classification\input\train",
                                                  batch_size=16, subset='validation',
                                                target_size=(224,224))
test_set = test_datagen.flow_from_directory(r"Ship Classification\input\test",
                                           batch_size=16,  target_size=(224,224))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Ship Classification\\input\\train'

In [22]:
from keras.layers import Dense,Flatten, Dropout
from keras.models import Model

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense

In [23]:
def create_model(input_shape, n_classes, optimizer='rmsprop'):
    conv_base = VGG16(include_top=False,
                     weights='imagenet',
                     input_shape=input_shape)
    for layer in conv_base.layers:
        layer.trainable = False

In [37]:
top_model = conv_base.output
top_model = Flatten(name="flatten")(top_model)
top_model = Dense(4096, activation='relu')(top_model)
top_model = Dense(1072, activation='relu')(top_model)
top_model = Dropout(0.2)(top_model)
output_layer = Dense(n_classes, activation='softmax')(top_model)

model = Model(inputs=conv_base.input, outputs=output_layer)

In [41]:
model.compile(optimizer=optimizer,
             loss='categorical_crossentropy',
             metrics=['accutacy'])

In [42]:
from tensorflow.keras.optimizers import Adam

In [43]:
input_shape = (224, 224, 3)
optim = Adam(learning_rate=0.001)
n_classes=5
vgg_model = create_model(input_shape, n_classes,optim)

In [45]:
vgg_model.summary()

NotImplementedError: Unimplemented `tf.keras.Model.call()`: if you intend to create a `Model` with the Functional API, please provide `inputs` and `outputs` arguments. Otherwise, subclass `Model` with an overridden `call()` method.